## Importamos librerias y se carga el CSV de las peliculas que luego procedermos a recomendar

In [120]:
import pandas as pd
import numpy as np
movies_df = pd.read_table('movies.dat', header=None, sep='::', names=['movie_id', 'movie_title', 'movie_genre'])
movies_df.head()

C:\Users\Jose\anaconda3\envs\platzi_data\lib\site-packages\pandas\io\parsers.py:755: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_csv(**locals())


,movie_id,movie_title,movie_genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### Cargar el CSV con el que sabemos el sentimiento del usuario para despues proceder a la recomendación de la película

In [121]:
sentiment_df = pd.read_csv('userSentiment.csv')
user = sentiment_df['user_name'][0]
sentiment_df

,user_id,user_name,status
0,1394615213199081475,MinPrac,positive


### Modificación del dataset
Se añaden columnas al dataset con cada uno de los géneros, dónde un 0 indica que si tiene ese género y un 1 que si es de ese género

In [122]:
# we convert the movie genres to a set of dummy variables
movies_df = pd.concat([movies_df, movies_df.movie_genre.str.get_dummies(sep='|')], axis=1)
movies_df.head()

,movie_id,movie_title,movie_genre,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Animation|Children's|Comedy,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [123]:
movie_categories = movies_df.columns[3:]
movies_df.loc[0]

movie_id                                 1
movie_title               Toy Story (1995)
movie_genre    Animation|Children's|Comedy
Action                                   0
Adventure                                0
Animation                                1
Children's                               1
Comedy                                   1
Crime                                    0
Documentary                              0
Drama                                    0
Fantasy                                  0
Film-Noir                                0
Horror                                   0
Musical                                  0
Mystery                                  0
Romance                                  0
Sci-Fi                                   0
Thriller                                 0
War                                      0
Western                                  0
Name: 0, dtype: object

### Se añade puntuación a los generos segun el estado de ánimo del usuario

In [124]:
from collections import OrderedDict
user_preferences = OrderedDict(zip(movie_categories, []))
status = sentiment_df.loc[0]['status']
if(status == 'positive'):
    user_preferences['Action'] = 1
    user_preferences['Adventure'] = 5
    user_preferences['Animation'] = 1
    user_preferences["Children's"] = 1
    user_preferences["Comedy"] = 15
    user_preferences['Crime'] = -5
    user_preferences['Documentary'] = 0
    user_preferences['Drama'] = -5
    user_preferences['Fantasy'] = 5
    user_preferences['Film-Noir'] = 0
    user_preferences['Horror'] = -2
    user_preferences['Musical'] = 1
    user_preferences['Mystery'] = -2
    user_preferences['Romance'] = 2
    user_preferences['Sci-Fi'] = -2
    user_preferences['War'] = -5
    user_preferences['Thriller'] = -5
    user_preferences['Western'] = 0
else:
    user_preferences['Action'] = 0
    user_preferences['Adventure'] = -2
    user_preferences['Animation'] = -2
    user_preferences["Children's"] = -2
    user_preferences["Comedy"] = -5
    user_preferences['Crime'] = 2
    user_preferences['Documentary'] = -5
    user_preferences['Drama'] = 15
    user_preferences['Fantasy'] = -5
    user_preferences['Film-Noir'] = 1
    user_preferences['Horror'] = 2
    user_preferences['Musical'] = -5
    user_preferences['Mystery'] = 3
    user_preferences['Romance'] = 1
    user_preferences['Sci-Fi'] = 5
    user_preferences['War'] = 3
    user_preferences['Thriller'] = 2
    user_preferences['Western'] =1  


In [125]:
def dot_product(vector_1, vector_2):
    return sum([ i*j for i,j in zip(vector_1, vector_2)])
def get_movie_score(movie_features, user_preferences):
    return dot_product(movie_features, user_preferences)

In [126]:
toy_story_features = movies_df.loc[0][movie_categories]
toy_story_features

Action         0
Adventure      0
Animation      1
Children's     1
Comedy         1
Crime          0
Documentary    0
Drama          0
Fantasy        0
Film-Noir      0
Horror         0
Musical        0
Mystery        0
Romance        0
Sci-Fi         0
Thriller       0
War            0
Western        0
Name: 0, dtype: object

In [127]:
toy_story_user_predicted_score = dot_product(toy_story_features, user_preferences.values())
toy_story_user_predicted_score

17

In [128]:
movies_df[movies_df.movie_title.str.contains('Die Hard')]

,movie_id,movie_title,movie_genre,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
163,165,Die Hard: With a Vengeance (1995),Action|Thriller,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1023,1036,Die Hard (1988),Action|Thriller,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1349,1370,Die Hard 2 (1990),Action|Thriller,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [129]:
die_hard_id = 1036
die_hard_features = movies_df[movies_df.movie_id==die_hard_id][movie_categories]
die_hard_features.T

,1023
Action,1
Adventure,0
Animation,0
Children's,0
Comedy,0
Crime,0
Documentary,0
Drama,0
Fantasy,0
Film-Noir,0


In [130]:
die_hard_user_predicted_score = dot_product(die_hard_features.values[0], user_preferences.values())
die_hard_user_predicted_score

-4

### Se recomiendan las 10 mejores películas para las preferencias del usuario

In [131]:
def get_movie_recommendations(user_preferences, n_recommendations):
    #we add a column to the movies_df dataset with the calculated score for each movie for the given user
    movies_df['score'] = movies_df[movie_categories].apply(get_movie_score,
                                                           args=([user_preferences.values()]), axis=1)
    return movies_df.sort_values(by=['score'], ascending=False)[['movie_title', 'movie_genre', 'score']][:n_recommendations]
    
print('Best 10 recommendations for ' + user + ':')   
get_movie_recommendations(user_preferences, 10)

Best 10 recommendations for MinPrac:


,movie_title,movie_genre,score
255,"Kid in King Arthur's Court, A (1995)",Adventure|Children's|Comedy|Fantasy|Romance,28
667,Space Jam (1996),Adventure|Animation|Children's|Comedy|Fantasy,27
1779,"Borrowers, The (1997)",Adventure|Children's|Comedy|Fantasy,26
1058,Willy Wonka and the Chocolate Factory (1971),Adventure|Children's|Comedy|Fantasy,26
1985,"Honey, I Shrunk the Kids (1989)",Adventure|Children's|Comedy|Fantasy|Sci-Fi,24
376,True Lies (1994),Action|Adventure|Comedy|Romance,23
2337,Romancing the Stone (1984),Action|Adventure|Comedy|Romance,23
1179,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance,23
1526,Hercules (1997),Adventure|Animation|Children's|Comedy|Musical,23
2336,"Jewel of the Nile, The (1985)",Action|Adventure|Comedy|Romance,23
